# Compile Snippy and BioHansel comparison results for individual genomes

The [nf-biohansel-snippy-comparison](https://github.com/peterk87/nf-biohansel-snippy-comparison) workflow was run using the list of 1000 SRA accessions present in [table S5](https://github.com/peterk87/biohansel-manuscript-supplementary-data/blob/master/Methods/BioHansel%20SNP%20detection%20validation/SalmonellaTyphi%20WGS%20dataset%20construction/Table_S5_List_of_1000_Salmonella_Typhi_Accessions.txt). 

The latest version of the workflow was retrieved with:

```bash
nextflow pull peterk87/nf-biohansel-snippy-comparison
```

After pulling the latest version of the workflow, it was necessary to modify `.nextflow/assets/peterk87/nf-biohansel-snippy-comparison/conf/base.config` so that the `errorStrategy` was `'retry'` (i.e. retry for all errors and failures)


```diff
diff --git a/conf/base.config b/conf/base.config
index 5fe7311..2c8fbfa 100644
--- a/conf/base.config
+++ b/conf/base.config
@@ -3,7 +3,7 @@ process {
   memory = { check_max( 1.GB * task.attempt, 'memory' ) }
   time = { check_max( 2.h * task.attempt, 'time' ) }

-  errorStrategy = { task.exitStatus in [143,137] ? 'retry' : 'finish' }
+  errorStrategy = 'retry' //{ task.exitStatus in [143,137] ? 'retry' : 'finish' }
   maxRetries = 3
   maxErrors = '-1'
```

The workflow was run with the following command:

```bash
nextflow run peterk87/nf-biohansel-snippy-comparison \
  -with-singularity nfbsc.sif \
  -resume -w work-bh-snippy-S2 \
  -profile slurm,singularity \
  --slurm_queue VALID_SLURM_PARTITION_NAME \
  --n_genomes 1000 \
  --schemesdir bh-snippy-comparison-schemes \
  --outdir nf-bh-snippy-results-S2
```

where `VALID_SLURM_PARTITION_NAME` was replaced with a valid Slurm partition/queue name on the cluster. 

There is one comparison for each genome so it's necessary to collect all the files for collation into one big table. 

First, the directory in which the comparison output files are stored is needed. 

In [1]:
from pathlib import Path

In [2]:
data_path = Path('/home/CSCScience.ca/pkruczkiewicz/K/pkruczkiewicz/nf-bh-snippy-results-S2/compare/')

Each comparison will be read into a Pandas DataFrame with a field added for genome name and scheme name derived from the input file name.

The tables will be concatenated into one big table `df_all`

In [3]:
import pandas as pd

In [4]:
dfs = []
for p in data_path.glob('*.csv'):
    df = pd.read_csv(p, index_col=0)
    genome, scheme = p.name.replace('.csv', '').split('-', maxsplit=1)
    df.sort_values('position', inplace=True)
    df['genome'] = genome
    df['scheme'] = scheme
    dfs.append(df)
df_all = pd.concat(dfs)

Preview of `df_all` table

In [5]:
df_all

,position,biohansel_A,biohansel_C,biohansel_G,biohansel_T,biohansel_depth,snippy_call,snippy_A,snippy_C,snippy_G,snippy_T,snippy_depth,genome,scheme
13,30191,0.0,59.0,0.0,0.0,59.0,C,0,73,0,0,73,ERR1791820,typhi_v1.2
17,102134,0.0,0.0,58.0,0.0,58.0,G,0,0,73,0,73,ERR1791820,typhi_v1.2
49,146672,0.0,0.0,55.0,0.0,55.0,G,0,0,76,0,76,ERR1791820,typhi_v1.2
50,183032,0.0,57.0,0.0,0.0,57.0,C,0,78,0,0,78,ERR1791820,typhi_v1.2
3,270119,0.0,40.0,0.0,0.0,40.0,C,0,57,0,0,57,ERR1791820,typhi_v1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,4288271,0.0,43.0,0.0,0.0,43.0,C,0,67,0,0,67,ERR1764671,typhi_v1.2
49,4355242,0.0,0.0,0.0,47.0,47.0,T,0,0,0,70,70,ERR1764671,typhi_v1.2
26,4388608,0.0,0.0,0.0,49.0,49.0,T,0,0,0,67,67,ERR1764671,typhi_v1.2
21,4602945,0.0,53.0,0.0,0.0,53.0,C,1,72,0,0,73,ERR1764671,typhi_v1.2


Mean biohansel kmer frequency for all 68 kmer markers and 1000 genomes

In [6]:
df_all.biohansel_depth.mean()

55.73157956685499

Mean Snippy SNV read mapping coverage depth

In [7]:
df_all.snippy_depth.mean()

76.87713672070376

Proportion of biohansel mean kmer frequency to Snippy mean read mapping coverage depth

In [8]:
df_all.biohansel_depth.mean() / df_all.snippy_depth.mean()

0.7249434870256547

Proportion of biohansel median kmer frequency to Snippy median read mapping coverage depth

In [9]:
df_all.biohansel_depth.median() / df_all.snippy_depth.median()

0.7246376811594203

Ensure that there are 1000 genome's worth of results

In [10]:
df_all.genome.unique().size

1000

Show instances where there are mixed biohansel results at 3X min kmer frequency

In [11]:
df_all[((df_all.loc[:,['biohansel_A', 'biohansel_G', 'biohansel_C', 'biohansel_T']] >= 3).sum(axis=1) > 1)]

,position,biohansel_A,biohansel_C,biohansel_G,biohansel_T,biohansel_depth,snippy_call,snippy_A,snippy_C,snippy_G,snippy_T,snippy_depth,genome,scheme
0,1215982,0.0,0.0,5.0,89.0,94.0,T,0,0,1,96,97,ERR279134,typhi_v1.2
2,1215982,0.0,0.0,5.0,83.0,88.0,T,0,1,0,98,99,ERR460297,typhi_v1.2
0,1215982,0.0,0.0,3.0,62.0,65.0,T,0,0,1,77,78,ERR331320,typhi_v1.2
2,1215982,0.0,0.0,3.0,51.0,54.0,T,0,0,0,69,69,ERR586911,typhi_v1.2
0,1840726,32.0,0.0,6.0,0.0,38.0,G,45,0,6,0,51,ERR467084,typhi_v1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,1215982,0.0,0.0,4.0,153.0,157.0,T,0,1,3,214,218,ERR204271,typhi_v1.2
2,3476113,0.0,3.0,0.0,61.0,64.0,T,0,4,0,70,74,ERR326637,typhi_v1.2
0,1215982,0.0,0.0,3.0,66.0,69.0,T,0,0,0,80,80,ERR460236,typhi_v1.2
3,1615349,0.0,68.0,0.0,4.0,72.0,C,0,89,0,5,94,ERR279182,typhi_v1.2


Show instances where there are mixed Snippy results at 3X min coverage

In [12]:
df_all[((df_all.loc[:,['snippy_A', 'snippy_G', 'snippy_C', 'snippy_T']] >= 3).sum(axis=1) > 1)]

,position,biohansel_A,biohansel_C,biohansel_G,biohansel_T,biohansel_depth,snippy_call,snippy_A,snippy_C,snippy_G,snippy_T,snippy_depth,genome,scheme
9,1215982,0.0,0.0,0.0,63.0,63.0,T,0,3,2,80,85,ERR314380,typhi_v1.2
1,146672,185.0,0.0,0.0,0.0,185.0,A,262,1,3,0,266,ERR1556249,typhi_v1.2
46,30191,0.0,39.0,0.0,0.0,39.0,C,5,64,0,0,69,ERR467084,typhi_v1.2
60,316185,0.0,50.0,0.0,0.0,50.0,C,7,73,0,0,80,ERR467084,typhi_v1.2
17,1215982,0.0,0.0,0.0,40.0,40.0,T,0,4,0,53,57,ERR467084,typhi_v1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,3476113,0.0,3.0,0.0,61.0,64.0,T,0,4,0,70,74,ERR326637,typhi_v1.2
3,1615349,0.0,68.0,0.0,4.0,72.0,C,0,89,0,5,94,ERR279182,typhi_v1.2
1,1780318,62.0,0.0,0.0,0.0,62.0,A,89,0,4,1,94,ERR279182,typhi_v1.2
13,3062269,0.0,64.0,0.0,0.0,64.0,C,0,87,0,3,90,ERR279182,typhi_v1.2


Show instances where there are mixed biohansel results at 6X min kmer frequency

In [13]:
df_all[((df_all.loc[:,['biohansel_A', 'biohansel_G', 'biohansel_C', 'biohansel_T']] >= 6).sum(axis=1) > 1)]

,position,biohansel_A,biohansel_C,biohansel_G,biohansel_T,biohansel_depth,snippy_call,snippy_A,snippy_C,snippy_G,snippy_T,snippy_depth,genome,scheme
0,1840726,32.0,0.0,6.0,0.0,38.0,G,45,0,6,0,51,ERR467084,typhi_v1.2
0,316488,0.0,6.0,0.0,67.0,73.0,T,0,7,0,85,92,ERR279137,typhi_v1.2
5,2737026,74.0,0.0,6.0,0.0,80.0,A,101,0,9,0,110,ERR279137,typhi_v1.2
4,3996716,0.0,6.0,0.0,97.0,103.0,T,0,10,0,130,140,ERR279137,typhi_v1.2
2,2342044,60.0,0.0,7.0,0.0,67.0,A,85,0,8,0,93,ERR279117,typhi_v1.2


Show instances where there are mixed Snippy results at 6X min coverage

In [14]:
df_all[((df_all.loc[:,['snippy_A', 'snippy_G', 'snippy_C', 'snippy_T']] >= 6).sum(axis=1) > 1)]

,position,biohansel_A,biohansel_C,biohansel_G,biohansel_T,biohansel_depth,snippy_call,snippy_A,snippy_C,snippy_G,snippy_T,snippy_depth,genome,scheme
60,316185,0.0,50.0,0.0,0.0,50.0,C,7,73,0,0,80,ERR467084,typhi_v1.2
0,1840726,32.0,0.0,6.0,0.0,38.0,G,45,0,6,0,51,ERR467084,typhi_v1.2
56,1934710,0.0,47.0,0.0,0.0,47.0,C,0,67,0,6,73,ERR467084,typhi_v1.2
40,30191,0.0,67.0,0.0,0.0,67.0,C,9,85,0,0,94,ERR279137,typhi_v1.2
0,316488,0.0,6.0,0.0,67.0,73.0,T,0,7,0,85,92,ERR279137,typhi_v1.2
39,1215982,0.0,0.0,0.0,96.0,96.0,T,0,7,0,110,117,ERR279137,typhi_v1.2
3,1615349,0.0,3.0,0.0,57.0,60.0,T,0,6,0,77,83,ERR279137,typhi_v1.2
1,1934710,0.0,54.0,0.0,3.0,57.0,C,0,73,0,6,79,ERR279137,typhi_v1.2
5,2737026,74.0,0.0,6.0,0.0,80.0,A,101,0,9,0,110,ERR279137,typhi_v1.2
6,3062269,0.0,5.0,0.0,62.0,67.0,T,0,7,0,79,86,ERR279137,typhi_v1.2


Show instances where there are mixed biohansel results at 8X min kmer frequency

In [15]:
df_all[((df_all.loc[:,['biohansel_A', 'biohansel_G', 'biohansel_C', 'biohansel_T']] >= 8).sum(axis=1) > 1)]

,position,biohansel_A,biohansel_C,biohansel_G,biohansel_T,biohansel_depth,snippy_call,snippy_A,snippy_C,snippy_G,snippy_T,snippy_depth,genome,scheme


What are the most frequent Snippy calls?

In [16]:
df_all.snippy_call.value_counts()

C    29936
G    26624
T     7061
A     4357
Name: snippy_call, dtype: int64

Summary stats of Snippy coverage depths at 68 SNV positions

In [17]:
df_all.snippy_depth.describe()

count    67978.000000
mean        76.877137
std         35.262588
min          4.000000
25%         56.000000
50%         69.000000
75%         86.000000
max        439.000000
Name: snippy_depth, dtype: float64

Summary stats of BioHansel kmer frequency at 68 SNV positions

In [18]:
df_all.biohansel_depth.describe()

count    67968.000000
mean        55.731580
std         25.656317
min          3.000000
25%         41.000000
50%         50.000000
75%         63.000000
max        293.000000
Name: biohansel_depth, dtype: float64

In [19]:
df_all.shape

(67978, 14)

## Fill in missing BioHansel/Snippy results

There are multiple missing BioHansel or Snippy results that need to be filled in and accounted for. To do that, a DataFrame (DF) will be constructed from each combination of genome to SNV position. This DF should have 68,000 rows.

The genome/SNV position DF will be merged (joined) with `df_all` and the missing values (`NaN` or `NA`) will be replaced with `0` for numeric fields. 

In [20]:
df_position_genome = pd.DataFrame([(p,g) for g in df_all.genome.unique() for p in df_all.position.unique()], columns=('position','genome'))

In [21]:
df_position_genome

,position,genome
0,30191,ERR1791820
1,102134,ERR1791820
2,146672,ERR1791820
3,183032,ERR1791820
4,270119,ERR1791820
...,...,...
67995,4288271,ERR1764671
67996,4355242,ERR1764671
67997,4388608,ERR1764671
67998,4602945,ERR1764671


In [22]:
df_68k = pd.merge(df_position_genome, df_all, how='left', on=['position', 'genome'])

In [23]:
df_68k

,position,genome,biohansel_A,biohansel_C,biohansel_G,biohansel_T,biohansel_depth,snippy_call,snippy_A,snippy_C,snippy_G,snippy_T,snippy_depth,scheme
0,30191,ERR1791820,0.0,59.0,0.0,0.0,59.0,C,0.0,73.0,0.0,0.0,73.0,typhi_v1.2
1,102134,ERR1791820,0.0,0.0,58.0,0.0,58.0,G,0.0,0.0,73.0,0.0,73.0,typhi_v1.2
2,146672,ERR1791820,0.0,0.0,55.0,0.0,55.0,G,0.0,0.0,76.0,0.0,76.0,typhi_v1.2
3,183032,ERR1791820,0.0,57.0,0.0,0.0,57.0,C,0.0,78.0,0.0,0.0,78.0,typhi_v1.2
4,270119,ERR1791820,0.0,40.0,0.0,0.0,40.0,C,0.0,57.0,0.0,0.0,57.0,typhi_v1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67995,4288271,ERR1764671,0.0,43.0,0.0,0.0,43.0,C,0.0,67.0,0.0,0.0,67.0,typhi_v1.2
67996,4355242,ERR1764671,0.0,0.0,0.0,47.0,47.0,T,0.0,0.0,0.0,70.0,70.0,typhi_v1.2
67997,4388608,ERR1764671,0.0,0.0,0.0,49.0,49.0,T,0.0,0.0,0.0,67.0,67.0,typhi_v1.2
67998,4602945,ERR1764671,0.0,53.0,0.0,0.0,53.0,C,1.0,72.0,0.0,0.0,73.0,typhi_v1.2


In [24]:
import numpy as np

In [25]:
bh_majority_call = df_68k['biohansel_A biohansel_C biohansel_G biohansel_T'.split()].apply(np.argmax, axis=1)

/home/CSCScience.ca/pkruczkiewicz/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


In [26]:
df_68k['biohansel_majority_call'] = bh_majority_call.str.replace(r'biohansel_', '')

In [27]:
(df_68k.biohansel_majority_call == df_68k.snippy_call).sum()

67965

In [28]:
(df_68k.biohansel_majority_call != df_68k.snippy_call).sum()

35

In [29]:
snippy_majority_call = df_68k['snippy_A snippy_C snippy_G snippy_T'.split()].apply(np.argmax, axis=1)

In [30]:
df_68k['snippy_majority_call'] = snippy_majority_call.str.replace(r'snippy_', '')

In [31]:
both_non_null = ~(pd.isnull(df_68k.biohansel_majority_call) | pd.isnull(df_68k.snippy_call))

In [32]:
both_non_null.sum()

67968

In [33]:
(df_68k.biohansel_majority_call[both_non_null] != df_68k.snippy_call[both_non_null]).sum()

3

In [34]:
(df_68k.biohansel_majority_call[both_non_null] != df_68k.snippy_majority_call[both_non_null]).sum()

0

## Merge Enterobase Sequencing Info

- Searched Enterobase for 1000 Typhi genomes by accession
- Output table of genome information
- Pulled accession, insert size, number of bases and average read length from column named `Data Source(Accession No.;Sequencing Platform;Sequencing Library;Insert Size;Experiment;Bases;Average Length;Status)` into a separate table
- Read seq info table tab-delimited file into DF
- Merged seq info DF into Snippy/BioHansel results DF

In [35]:
df_seq_platform_info = pd.read_table('typhi-1000-enterobase-seq-platform-info-S2.tsv')

In [36]:
df_68k_seqinfo = pd.merge(df_seq_platform_info, df_68k, on='genome', how='right')

In [37]:
df_68k_seqinfo.read_length.describe()

count    67252.000000
mean       108.430738
std         13.880588
min         96.000000
25%        100.000000
50%        100.000000
75%        125.000000
max        250.000000
Name: read_length, dtype: float64

In [38]:
df_68k_seqinfo.snippy_depth.fillna(0, inplace=True)

In [39]:
df_68k_seqinfo.biohansel_depth.fillna(0, inplace=True)

## Fill in null values for missing results

- Where `(snippy|biohansel)_(A|C|G|T)` is null/NaN, replace with 0.
- Where `(snippy|biohansel)_depth` is null/NaN, replace with 0.

In [40]:
for x in ['snippy', 'biohansel']:
    for y in list('ACGT'):
        df_68k_seqinfo[f'{x}_{y}'].fillna(0, inplace=True)

In [41]:
for x in ['snippy', 'biohansel']:
    df_68k_seqinfo[f'{x}_depth'].fillna(0, inplace=True)

## Add columns for number of Snippy and BioHansel calls at 3X, 6X, 8X

In [42]:
for t in [3, 6, 8]:
    for x in ['snippy', 'biohansel']:
        cols = [f'{x}_{y}' for y in list('ACGT')]
        gte_t = (df_68k_seqinfo[cols] >= t).sum(axis=1)
        df_68k_seqinfo[f'{x}_bases_gte_{t}X'] = gte_t
        print(t, x)

3 snippy
3 biohansel
6 snippy
6 biohansel
8 snippy
8 biohansel


In [43]:
(df_68k_seqinfo.snippy_bases_gte_3X > 1).sum()

92

In [44]:
(df_68k_seqinfo.snippy_bases_gte_3X == 0).sum()

22

In [45]:
(df_68k_seqinfo.snippy_bases_gte_6X > 1).sum()

19

In [46]:
(df_68k_seqinfo.snippy_bases_gte_6X == 0).sum()

23

In [47]:
(df_68k_seqinfo.snippy_bases_gte_8X > 1).sum()

4

In [48]:
(df_68k_seqinfo.snippy_bases_gte_8X == 0).sum()

24

In [49]:
df_68k_seqinfo.shape

(68000, 26)

## BioHansel and Snippy null and mixed result counts

In [50]:
d = {'both_null': {},
     'both_mixed': {}}
for x in ['snippy', 'biohansel']:
    tmp_null = {}
    tmp_mixed = {}
    d[f'{x}_null'] = tmp_null
    d[f'{x}_mixed'] = tmp_mixed
    for t in [3, 6, 8]:
        tmp_null[t] = (df_68k_seqinfo[f'{x}_bases_gte_{t}X'] == 0).sum()
        tmp_mixed[t] = (df_68k_seqinfo[f'{x}_bases_gte_{t}X'] > 1).sum()
        d['both_null'][t] = ((df_68k_seqinfo[f'snippy_bases_gte_{t}X'] == 0) & (df_68k_seqinfo[f'biohansel_bases_gte_{t}X'] == 0)).sum()
        d['both_mixed'][t] = ((df_68k_seqinfo[f'snippy_bases_gte_{t}X']>1) & (df_68k_seqinfo[f'biohansel_bases_gte_{t}X'] > 1)).sum()

In [51]:
pd.DataFrame(d)

,both_null,both_mixed,snippy_null,snippy_mixed,biohansel_null,biohansel_mixed
3,22,43,22,92,32,72
6,23,5,23,19,34,5
8,24,0,24,4,35,0


Filter for results with no Snippy coverage

In [52]:
df_68k_seqinfo[df_68k_seqinfo.snippy_depth == 0]

,genome,insert_size,run_accession,bases,read_length,position,biohansel_A,biohansel_C,biohansel_G,biohansel_T,...,snippy_depth,scheme,biohansel_majority_call,snippy_majority_call,snippy_bases_gte_3X,biohansel_bases_gte_3X,snippy_bases_gte_6X,biohansel_bases_gte_6X,snippy_bases_gte_8X,biohansel_bases_gte_8X
8860,ERR460327,227,ERX426668,423108576.0,99.0,1780318,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0,0,0,0,0,0
9676,ERR460349,227,ERX426690,515525670.0,99.0,1780318,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0,0,0,0,0,0
12056,ERR460335,227,ERX426676,485333640.0,99.0,1780318,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0,0,0,0,0,0
16180,ERR360426,203,ERX333197,291851800.0,100.0,4355242,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0,0,0,0,0,0
18357,ERR360489,201,ERX333260,243621400.0,100.0,4388608,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0,0,0,0,0,0
43070,ERR279352,215,ERX252685,654736200.0,100.0,1934710,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0,0,0,0,0,0
49660,ERR1764817,450,ERX1830386,317295250.0,125.0,1780318,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0,0,0,0,0,0
49728,ERR1764815,450,ERX1830384,311245000.0,125.0,1780318,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0,0,0,0,0,0
49864,ERR1764811,450,ERX1830380,325828000.0,125.0,1780318,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0,0,0,0,0,0
50612,ERR1764792,450,ERX1830361,336335000.0,125.0,1780318,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0,0,0,0,0,0


## Format DF for output

- rename columns
- adjust position from 0-base to 1-base indexing

In [53]:
df_68k_seqinfo.columns

Index(['genome', 'insert_size', 'run_accession', 'bases', 'read_length',
       'position', 'biohansel_A', 'biohansel_C', 'biohansel_G', 'biohansel_T',
       'biohansel_depth', 'snippy_call', 'snippy_A', 'snippy_C', 'snippy_G',
       'snippy_T', 'snippy_depth', 'scheme', 'biohansel_majority_call',
       'snippy_majority_call', 'snippy_bases_gte_3X', 'biohansel_bases_gte_3X',
       'snippy_bases_gte_6X', 'biohansel_bases_gte_6X', 'snippy_bases_gte_8X',
       'biohansel_bases_gte_8X'],
      dtype='object')

In [54]:
df_68k_seqinfo.rename(
    columns=dict(
        biohansel_majority_call='biohansel_majority_base_before_QC',
        snippy_call='snippy_consensus_sequence_call',
        snippy_majority_call='snippy_majority_base_before_QC'
    ), inplace=True)

In [55]:
df_68k_seqinfo.position = df_68k_seqinfo.position + 1

Output DF to CSV file

In [56]:
df_68k_seqinfo.to_csv('bh-snippy-compare-1000genomes-S2-accessions.csv')

## Pearson correlation between different values

In [57]:
from scipy.stats import pearsonr

In [58]:
pearsonr(df_68k_seqinfo.biohansel_depth, df_68k_seqinfo.snippy_depth)

(0.9742715767083514, 0.0)

Is there a correlation between read length and BioHansel SNV kmer frequency?

In [59]:
non_null_read_length_mask = ~pd.isnull(df_68k_seqinfo.read_length)
pearsonr(df_68k_seqinfo.biohansel_depth[non_null_read_length_mask], df_68k_seqinfo.read_length[non_null_read_length_mask])

(0.028430127656757127, 1.654146151884857e-13)

Is there a correlation between read length and Snippy SNV read mapping coverage depth?

In [60]:
non_null_read_length_mask = ~pd.isnull(df_68k_seqinfo.read_length)
pearsonr(df_68k_seqinfo.snippy_depth[non_null_read_length_mask], df_68k_seqinfo.read_length[non_null_read_length_mask])

(-0.027823792503598205, 5.322343155834001e-13)